In [15]:
import MDAnalysis
import numpy as np
u = MDAnalysis.Universe("bound.psf", "bound.pdb")

In [16]:
"""
Mimic the VMD command "measure minmax"
"""
def measure_minmax(atom_positions):
    xyz_array = np.transpose(atom_positions)
    min_x = np.min(xyz_array[0])
    max_x = np.max(xyz_array[0])
    min_y = np.min(xyz_array[1])
    max_y = np.max(xyz_array[1])
    min_z = np.min(xyz_array[2])
    max_z = np.max(xyz_array[2])
    return np.array([[min_x, min_y, min_z],[max_x, max_y, max_z]])


"""
Mimic the VMD command "measure center"
"""
def measure_center(atom_positions):
    xyz_array = np.transpose(atom_positions)
    center_x = np.average(xyz_array[0])
    center_y = np.average(xyz_array[1])
    center_z = np.average(xyz_array[2])
    return np.array([center_x, center_y, center_z])


def get_cell(atom_positions):
    minmax_array = measure_minmax(atom_positions)
    vec = minmax_array[1] - minmax_array[0]
    cell_basis_vector1 = np.array([vec[0], 0, 0])
    cell_basis_vector2 = np.array([0, vec[1], 0])
    cell_basis_vector3 = np.array([0, 0, vec[2]])
    return np.array([cell_basis_vector1,
                     cell_basis_vector2,
                     cell_basis_vector3])


all_atoms = u.select_atoms("all")
dist_array = all_atoms.positions
print(measure_center(dist_array))
print(get_cell(dist_array))

[ 0.16814499 -0.38182235 -0.6636063 ]
[[49.47299957  0.          0.        ]
 [ 0.         49.5359993   0.        ]
 [ 0.          0.         49.49700165]]


In [17]:
# access the current timestep
u.trajectory[0].triclinic_dimensions = get_cell(dist_array)

In [18]:
all_atoms.write("p41_gmx.gro")

In [22]:
# extract the protein to the gromacs index file
protein = u.select_atoms("segid SH3D and not (name H*)")
protein.write("protein.ndx", name="Protein_SH3D")

In [21]:
# extract the ligand to the gromacs index file
ligand = u.select_atoms("segid PPRO and not (name H*)")
ligand.write("ligand.ndx", name="Ligand_PPRO")

In [23]:
# merge the index files into a single one
def merge_files(filename_list, output_filename):
    with open(output_filename, "w") as foutput:
        for fn in filename_list:
            with open(fn, "r") as finput:
                for line in finput:
                    foutput.write(line)

merge_files(["protein.ndx", "ligand.ndx"], "colvars.ndx")